In [ ]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%run ./tools/test_net.py --gpu 0 --def models/pascal_voc/ResNet-50/rfcn_end2end/test_agnostic.prototxt --net /home/ubuntu/py-R-FCN/output/rfcn_end2end_ohem/voc_2007_trainval/resnet50_rfcn_ohem_iter_20000.caffemodel --imdb voc_2007_test --cfg experiments/cfgs/rfcn_end2end_ohem.yml

In [ ]:
import cPickle  
with open('output/rfcn_end2end_ohem/voc_2007_test/resnet50_rfcn_ohem_iter_20000/test_preds.pkl','rb') as f:
    test_train_preds = cPickle.load(f)  
    
test_preds = test_train_preds[:1000]
train_preds = test_train_preds[1000:,:]

In [ ]:
import cPickle  
with open('output/rfcn_end2end_ohem/voc_2007_test/resnet50_rfcn_ohem_iter_20000/detections_full.pkl','rb') as f:
    detections_full = cPickle.load(f) 
    
CONF_THRESH = 0.8
outputs = []

for im in range(len(detections_full[0])):
    outputs_im = []
    for cls in range(1,len(FISH_CLASSES)):
        detects = detections_full[cls][im]
        for i in range(len(detects)):
            if np.max(detects[i,5:]) >= CONF_THRESH:
                outputs_im.append(detects[i,4:])
    if len(outputs_im) == 0:
        outputs_im.append(test_train_preds[im])
    outputs_im = np.asarray(outputs_im)
    outputs_im = np.mean(outputs_im, axis=0)
    outputs.append(outputs_im)
outputs = np.asarray(outputs)

test_preds = outputs[:1000]
train_preds = outputs[1000:,:]

In [ ]:
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG','OTHER', 'SHARK', 'YFT']

with open("./data/VOCdevkit2007/VOC2007/ImageSets/Main/test.txt","r") as f:
    ims = f.readlines()
test_files = [im[:-1]+'.jpg' for im in ims[:1000]]
train_files = [im[:-1] for im in ims[1000:]]
print(len(train_files))
with open("./data/VOCdevkit2007/VOC2007/ImageSets/Main/train_test.txt","r") as f:
    train_file_labels = f.readlines()

log_losses = []
for i in range(len(train_preds)):
    im = train_files[i]
    for im_label in train_file_labels:
        if im == im_label[:9]:
            label = im_label[10:-1]
            index = FISH_CLASSES.index(label)
            log_losses.append(-math.log(train_preds[i,index]))
log_loss = sum(log_losses) / float(len(log_losses))
print('logloss of train is', log_loss )

In [ ]:
import datetime

submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
#submission.loc[:, 'image'] = pd.Series(test_files, index=submission.index)
submission.insert(0, 'image', test_files)

now = datetime.datetime.now()
info = 'RFCN_' + '{:.4f}'.format(log_loss)
sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
submission.to_csv(sub_file, index=False)

In [ ]:
import cPickle  
with open('output/rfcn_end2end_ohem/voc_2007_test/resnet50_rfcn_ohem_iter_20000/detections.pkl','rb') as f:
    detections = cPickle.load(f)  